*Import libraries*

In [1]:
from pymongo import MongoClient, TEXT, ASCENDING
from datetime import datetime, timedelta

*Connect to mongodb*

In [2]:
# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017")
db = client["eduhub_db"]

*1: IMPLEMENT TEXT SEARCH FUCTIONALITY FOR COURSE CONTENT*

In [3]:
#IMPLEMENT TEXT SEARCH FUCTIONALITY FOR COURSE CONTENT


# 1. Text Search on Courses
# Create text index on course title and description
db.courses.create_index([("title", TEXT), ("description", TEXT)])

# Search courses using a keyword
keyword = "python"
results = db.courses.find({"$text": {"$search": keyword}})
print(f"\n Courses matching '{keyword}':")
for course in results:
    print(f"- {course['title']}")     


 Courses matching 'python':
- Python for Beginners
- Python for Beginners
- Data Engineering with Python
- Data Engineering with Python


*2: CREATE A RECOMMENDATION SYSTEM USING AGREGATION*

In [6]:
# 2. Simple Recommendation System (Based on Category)

def recommend_courses_for_user(user_id):
    # Get courses the user is enrolled in
    enrolled_course_ids = db.enrollments.find({"student_id": user_id})
    course_ids = [e["course_id"] for e in enrolled_course_ids]

    # Get categories of enrolled courses
    enrolled_courses = db.courses.find({"_id": {"$in": course_ids}})
    categories = [c["category"] for c in enrolled_courses]

    # Recommend other courses in same categories
    recommendations = db.courses.find({
        "category": {"$in": categories},
        "_id": {"$nin": course_ids}
    })

    print(f"\n Recommended courses for user {user_id}:")
    for rec in recommendations:
        print(f"- {rec['title']} ({rec['category']})")

# Example usage
recommend_courses_for_user("user123")



 Recommended courses for user user123:


*3: DESIGN A DATA ARCHIVING STRATEGY FOR OLD ENROLLMENTS*

In [10]:
# 3. Archiving Old Enrollments (> 1 year old)

one_year_ago = datetime.utcnow() - timedelta(days=60)

# Move to archived collection
old_enrollments = db.enrollments.find({"enrolled_at": {"$lt": one_year_ago}})
archived = [e for e in old_enrollments]

if archived:
    db.archived_enrollments.insert_many(archived)
    db.enrollments.delete_many({"enrolled_at": {"$lt": one_year_ago}})
    print(f"\n Archived {len(archived)} old enrollments.")
else:
    print("\n No enrollments to archive.")


 No enrollments to archive.


C:\Users\hello\AppData\Local\Temp\ipykernel_13812\2090906355.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  one_year_ago = datetime.utcnow() - timedelta(days=60)


*4: IMPLEMENT GEOSPARTIAL QUERIES FOR LOCATION_BASED COURSE RECOMENDATION*

In [11]:
# 4. Geospatial Queries (Find Courses Near a Student)
# Add 2dsphere index for location field
db.courses.create_index([("location", "2dsphere")])

# Example: Find courses within 5km of student's location
student_location = {
    "type": "Point",
    "coordinates": [3.3792, 6.5244]  # Example: Lagos
}

nearby_courses = db.courses.find({
    "location": {
        "$near": {
            "$geometry": student_location,
            "$maxDistance": 5000  # 5 kilometers
        }
    }
})

print("\n📍 Nearby courses (within 5km):")
for course in nearby_courses:
    print(f"- {course['title']}")


📍 Nearby courses (within 5km):
